# Polynomial Regression Tutorial
## Capturing Non-Linear Relationships

Welcome to **Polynomial Regression** - where we go beyond straight lines!

### What you'll learn:
- When linear regression isn't enough
- How to create polynomial features
- Finding the optimal polynomial degree
- Bias-variance tradeoff and overfitting
- Cross-validation for model selection
- Regularization techniques

### Our Challenge:
Model house prices with **curved relationships** that simple linear regression can't capture.

Let's capture those curves! 📈

## Step 1: Import Libraries and Load Data

For polynomial regression, we need additional tools for feature transformation and model validation.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, validation_curve
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Load the dataset
data = pd.read_csv('dataset.csv')

print("✅ Libraries imported and data loaded!")
print(f"Dataset shape: {data.shape}")
print(f"Columns: {list(data.columns)}")
data.head()

## Step 2: Explore Non-Linear Relationships

Let's investigate potential non-linear patterns in our data that polynomial regression could capture.

In [ ]:
# Create scatter plots to identify non-linear relationships
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Exploring Non-Linear Relationships', fontsize=16, fontweight='bold')

# Area vs Price
axes[0, 0].scatter(data['area'], data['price'], alpha=0.6, color='blue')
axes[0, 0].set_xlabel('Area (sq ft)')
axes[0, 0].set_ylabel('Price ($)')
axes[0, 0].set_title('Area vs Price')
axes[0, 0].grid(True, alpha=0.3)

# Age vs Price (might show non-linear depreciation)
axes[0, 1].scatter(data['age'], data['price'], alpha=0.6, color='green')
axes[0, 1].set_xlabel('Age (years)')
axes[0, 1].set_ylabel('Price ($)')
axes[0, 1].set_title('Age vs Price')
axes[0, 1].grid(True, alpha=0.3)

# Bedrooms vs Price
axes[1, 0].scatter(data['bedrooms'], data['price'], alpha=0.6, color='red')
axes[1, 0].set_xlabel('Bedrooms')
axes[1, 0].set_ylabel('Price ($)')
axes[1, 0].set_title('Bedrooms vs Price')
axes[1, 0].grid(True, alpha=0.3)

# Create a synthetic non-linear feature: Area squared relationship
area_squared = data['area'] ** 2
axes[1, 1].scatter(area_squared, data['price'], alpha=0.6, color='purple')
axes[1, 1].set_xlabel('Area Squared (sq ft)²')
axes[1, 1].set_ylabel('Price ($)')
axes[1, 1].set_title('Area² vs Price')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate correlations with polynomial features
print("=" * 50)
print("CORRELATION ANALYSIS")
print("=" * 50)

correlations = {
    'area (linear)': np.corrcoef(data['area'], data['price'])[0, 1],
    'area² (quadratic)': np.corrcoef(data['area']**2, data['price'])[0, 1],
    'area³ (cubic)': np.corrcoef(data['area']**3, data['price'])[0, 1],
    'age (linear)': np.corrcoef(data['age'], data['price'])[0, 1],
    'age² (quadratic)': np.corrcoef(data['age']**2, data['price'])[0, 1]
}

for feature, corr in correlations.items():
    print(f"{feature:20s}: {corr:7.4f}")

# Find the strongest correlation
best_feature = max(correlations.items(), key=lambda x: abs(x[1]))
print(f"\n🎯 Strongest correlation: {best_feature[0]} ({best_feature[1]:.4f})")

## Step 3: Data Preparation

Prepare our data for polynomial regression, focusing on the most promising features.

In [ ]:
# Prepare data for polynomial regression
print("=" * 50)
print("DATA PREPARATION")
print("=" * 50)

# Handle categorical variables
label_encoder = LabelEncoder()
data_processed = data.copy()
data_processed['location_encoded'] = label_encoder.fit_transform(data['location'])

# Select features
feature_columns = ['area', 'bedrooms', 'age', 'location_encoded']
X = data_processed[feature_columns]
y = data_processed['price']

print(f"Features: {feature_columns}")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\nTraining samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")
print("✅ Data preparation complete!")

## Step 4: Linear Regression Baseline

First, let's establish a baseline with simple linear regression to compare against.

In [ ]:
# Train linear regression baseline
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Make predictions
y_pred_linear = linear_model.predict(X_test)

# Evaluate baseline
mse_linear = mean_squared_error(y_test, y_pred_linear)
rmse_linear = np.sqrt(mse_linear)
r2_linear = r2_score(y_test, y_pred_linear)
mae_linear = mean_absolute_error(y_test, y_pred_linear)

print("=" * 50)
print("LINEAR REGRESSION BASELINE")
print("=" * 50)
print(f"Mean Squared Error (MSE):  ${mse_linear:,.2f}")
print(f"Root Mean Squared Error:   ${rmse_linear:,.2f}")
print(f"Mean Absolute Error:       ${mae_linear:,.2f}")
print(f"R² Score:                  {r2_linear:.4f}")
print(f"Explained Variance:        {r2_linear*100:.1f}%")

# Store baseline results for comparison
baseline_results = {
    'model_name': 'Linear Regression',
    'degree': 1,
    'mse': mse_linear,
    'rmse': rmse_linear,
    'r2': r2_linear,
    'mae': mae_linear
}

print("\n✅ Baseline established!")

## Step 5: Polynomial Feature Creation

Transform our features into polynomial features of different degrees.

In [ ]:
# Explore different polynomial degrees
degrees = [2, 3, 4, 5]
polynomial_results = []

print("=" * 50)
print("POLYNOMIAL FEATURE EXPLORATION")
print("=" * 50)

for degree in degrees:
    print(f"\n🔍 Testing Polynomial Degree {degree}:")
    
    # Create polynomial features
    poly_features = PolynomialFeatures(degree=degree, include_bias=False)
    X_train_poly = poly_features.fit_transform(X_train)
    X_test_poly = poly_features.transform(X_test)
    
    print(f"  Original features: {X_train.shape[1]}")
    print(f"  Polynomial features: {X_train_poly.shape[1]}")
    print(f"  Feature expansion: {X_train_poly.shape[1] / X_train.shape[1]:.1f}x")
    
    # Train polynomial regression
    poly_model = LinearRegression()
    poly_model.fit(X_train_poly, y_train)
    
    # Make predictions
    y_pred_poly = poly_model.predict(X_test_poly)
    
    # Evaluate
    mse_poly = mean_squared_error(y_test, y_pred_poly)
    rmse_poly = np.sqrt(mse_poly)
    r2_poly = r2_score(y_test, y_pred_poly)
    mae_poly = mean_absolute_error(y_test, y_pred_poly)
    
    print(f"  RMSE: ${rmse_poly:,.2f}")
    print(f"  R²: {r2_poly:.4f}")
    
    # Store results
    polynomial_results.append({
        'model_name': f'Polynomial (degree {degree})',
        'degree': degree,
        'mse': mse_poly,
        'rmse': rmse_poly,
        'r2': r2_poly,
        'mae': mae_poly,
        'num_features': X_train_poly.shape[1],
        'model': poly_model,
        'poly_features': poly_features
    })
    
    # Check for improvement
    improvement = r2_poly - r2_linear
    if improvement > 0:
        print(f"  ✅ Improvement: +{improvement:.4f} R²")
    else:
        print(f"  ❌ Worse: {improvement:.4f} R²")

print("\n✅ Polynomial exploration complete!")

## Step 6: Cross-Validation for Model Selection

Use cross-validation to find the optimal polynomial degree and avoid overfitting.

In [ ]:
# Perform cross-validation for different polynomial degrees
degrees_cv = range(1, 8)
cv_scores_mean = []
cv_scores_std = []
train_scores_mean = []

print("=" * 50)
print("CROSS-VALIDATION FOR MODEL SELECTION")
print("=" * 50)

X_combined = pd.concat([X_train, X_test])
y_combined = pd.concat([y_train, y_test])

for degree in degrees_cv:
    # Create pipeline with polynomial features and linear regression
    pipeline = Pipeline([
        ('poly', PolynomialFeatures(degree=degree, include_bias=False)),
        ('scaler', StandardScaler()),
        ('linear', LinearRegression())
    ])
    
    # Cross-validation scores (validation)
    cv_scores = cross_val_score(pipeline, X_combined, y_combined, 
                               cv=5, scoring='r2')
    cv_scores_mean.append(cv_scores.mean())
    cv_scores_std.append(cv_scores.std())
    
    # Training scores (to detect overfitting)
    pipeline.fit(X_combined, y_combined)
    train_score = pipeline.score(X_combined, y_combined)
    train_scores_mean.append(train_score)
    
    print(f"Degree {degree}: CV R² = {cv_scores.mean():.4f} (±{cv_scores.std():.4f}), "
          f"Train R² = {train_score:.4f}")

# Find optimal degree
optimal_degree = degrees_cv[np.argmax(cv_scores_mean)]
best_cv_score = max(cv_scores_mean)

print(f"\n🎯 Optimal Degree: {optimal_degree}")
print(f"🎯 Best CV R²: {best_cv_score:.4f}")

# Plot validation curves
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(degrees_cv, cv_scores_mean, 'o-', color='blue', label='Validation Score')
plt.fill_between(degrees_cv, 
                 np.array(cv_scores_mean) - np.array(cv_scores_std),
                 np.array(cv_scores_mean) + np.array(cv_scores_std),
                 alpha=0.2, color='blue')
plt.plot(degrees_cv, train_scores_mean, 'o-', color='red', label='Training Score')
plt.axvline(x=optimal_degree, color='green', linestyle='--', label=f'Optimal Degree ({optimal_degree})')
plt.xlabel('Polynomial Degree')
plt.ylabel('R² Score')
plt.title('Validation Curve: Polynomial Degree vs Performance')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot overfitting detection
plt.subplot(1, 2, 2)
gap = np.array(train_scores_mean) - np.array(cv_scores_mean)
plt.plot(degrees_cv, gap, 'o-', color='orange', linewidth=2)
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.axvline(x=optimal_degree, color='green', linestyle='--', label=f'Optimal Degree ({optimal_degree})')
plt.xlabel('Polynomial Degree')
plt.ylabel('Training R² - Validation R²')
plt.title('Overfitting Detection')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Analyze overfitting
print(f"\n📊 Overfitting Analysis:")
for i, degree in enumerate(degrees_cv):
    gap = train_scores_mean[i] - cv_scores_mean[i]
    if gap < 0.05:
        status = "✅ Good fit"
    elif gap < 0.1:
        status = "⚠️ Slight overfitting"
    else:
        status = "❌ Overfitting"
    print(f"  Degree {degree}: Gap = {gap:.4f} - {status}")

## Step 7: Train Final Polynomial Model

Train the final model using the optimal polynomial degree.

In [ ]:
# Train the final polynomial model with optimal degree
print("=" * 50)
print(f"FINAL POLYNOMIAL MODEL (DEGREE {optimal_degree})")
print("=" * 50)

# Create polynomial features with optimal degree
final_poly_features = PolynomialFeatures(degree=optimal_degree, include_bias=False)
X_train_final = final_poly_features.fit_transform(X_train)
X_test_final = final_poly_features.transform(X_test)

print(f"Feature transformation:")
print(f"  Original features: {X_train.shape[1]}")
print(f"  Polynomial features: {X_train_final.shape[1]}")
print(f"  Feature names (first 10): {final_poly_features.get_feature_names_out(feature_columns)[:10]}")

# Scale features for better numerical stability
scaler = StandardScaler()
X_train_final_scaled = scaler.fit_transform(X_train_final)
X_test_final_scaled = scaler.transform(X_test_final)

# Train final model
final_model = LinearRegression()
final_model.fit(X_train_final_scaled, y_train)

# Make predictions
y_pred_final = final_model.predict(X_test_final_scaled)

# Evaluate final model
mse_final = mean_squared_error(y_test, y_pred_final)
rmse_final = np.sqrt(mse_final)
r2_final = r2_score(y_test, y_pred_final)
mae_final = mean_absolute_error(y_test, y_pred_final)

print(f"\n📊 Final Model Performance:")
print(f"  Mean Squared Error:   ${mse_final:,.2f}")
print(f"  Root Mean Squared Error: ${rmse_final:,.2f}")
print(f"  Mean Absolute Error:  ${mae_final:,.2f}")
print(f"  R² Score:             {r2_final:.4f}")
print(f"  Explained Variance:   {r2_final*100:.1f}%")

# Compare with baseline
improvement = r2_final - r2_linear
print(f"\n🎯 Improvement over Linear Regression:")
print(f"  R² improvement: +{improvement:.4f} ({improvement/r2_linear*100:+.1f}%)")
print(f"  RMSE improvement: ${rmse_linear - rmse_final:,.2f}")

if improvement > 0.01:
    print("  ✅ Significant improvement with polynomial features!")
elif improvement > 0:
    print("  👍 Modest improvement with polynomial features.")
else:
    print("  ❌ No improvement - linear model is sufficient.")

print("\n✅ Final model trained successfully!")

## Step 8: Feature Importance Analysis

Analyze which polynomial features contribute most to our predictions.

In [ ]:
# Analyze feature importance
print("=" * 50)
print("POLYNOMIAL FEATURE IMPORTANCE")
print("=" * 50)

# Get feature names and coefficients
feature_names = final_poly_features.get_feature_names_out(feature_columns)
coefficients = final_model.coef_

# Create feature importance (absolute coefficients)
feature_importance = list(zip(feature_names, np.abs(coefficients)))
feature_importance.sort(key=lambda x: x[1], reverse=True)

print(f"Top 10 Most Important Features:")
for i, (feature, importance) in enumerate(feature_importance[:10], 1):
    print(f"  {i:2d}. {feature:25s}: {importance:10.2f}")

# Visualize top features
top_features = feature_importance[:15]
features, importances = zip(*top_features)

plt.figure(figsize=(12, 8))
bars = plt.barh(range(len(features)), importances, alpha=0.7)
plt.yticks(range(len(features)), features)
plt.xlabel('Coefficient Magnitude')
plt.title(f'Top 15 Polynomial Feature Importance (Degree {optimal_degree})')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3)

# Color bars by feature type
for i, bar in enumerate(bars):
    if 'area' in features[i]:
        bar.set_color('blue')
    elif 'age' in features[i]:
        bar.set_color('red')
    elif 'bedrooms' in features[i]:
        bar.set_color('green')
    else:
        bar.set_color('orange')

plt.tight_layout()
plt.show()

# Analyze feature types
feature_types = {'area': 0, 'age': 0, 'bedrooms': 0, 'location': 0, 'interaction': 0}

for feature, importance in feature_importance:
    if feature.count(' ') > 0:  # Interaction terms
        feature_types['interaction'] += importance
    elif 'area' in feature:
        feature_types['area'] += importance
    elif 'age' in feature:
        feature_types['age'] += importance
    elif 'bedrooms' in feature:
        feature_types['bedrooms'] += importance
    elif 'location' in feature:
        feature_types['location'] += importance

print(f"\n📊 Feature Type Importance:")
for feature_type, total_importance in sorted(feature_types.items(), 
                                           key=lambda x: x[1], reverse=True):
    print(f"  {feature_type:12s}: {total_importance:8.2f}")

## Step 9: Visualize Results

Create comprehensive visualizations to compare linear vs polynomial regression.

In [ ]:
# Create comprehensive visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Polynomial vs Linear Regression Comparison', fontsize=16, fontweight='bold')

# 1. Actual vs Predicted (Linear)
axes[0, 0].scatter(y_test, y_pred_linear, alpha=0.6, color='blue')
axes[0, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
                'r--', linewidth=2, label='Perfect Prediction')
axes[0, 0].set_xlabel('Actual Price ($)')
axes[0, 0].set_ylabel('Predicted Price ($)')
axes[0, 0].set_title(f'Linear Regression (R² = {r2_linear:.3f})')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Actual vs Predicted (Polynomial)
axes[0, 1].scatter(y_test, y_pred_final, alpha=0.6, color='green')
axes[0, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
                'r--', linewidth=2, label='Perfect Prediction')
axes[0, 1].set_xlabel('Actual Price ($)')
axes[0, 1].set_ylabel('Predicted Price ($)')
axes[0, 1].set_title(f'Polynomial Regression (R² = {r2_final:.3f})')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Residuals Comparison
residuals_linear = y_test - y_pred_linear
residuals_poly = y_test - y_pred_final

axes[0, 2].scatter(y_pred_linear, residuals_linear, alpha=0.6, color='blue', label='Linear')
axes[0, 2].scatter(y_pred_final, residuals_poly, alpha=0.6, color='green', label='Polynomial')
axes[0, 2].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0, 2].set_xlabel('Predicted Price ($)')
axes[0, 2].set_ylabel('Residuals ($)')
axes[0, 2].set_title('Residuals Analysis')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# 4. Model Complexity vs Performance
all_results = [baseline_results] + polynomial_results
degrees_plot = [result['degree'] for result in all_results]
r2_scores = [result['r2'] for result in all_results]
num_features = [X_train.shape[1] if result['degree'] == 1 
                else result['num_features'] for result in all_results]

axes[1, 0].plot(degrees_plot, r2_scores, 'o-', linewidth=2, markersize=8)
axes[1, 0].axvline(x=optimal_degree, color='red', linestyle='--', 
                  label=f'Optimal Degree ({optimal_degree})')
axes[1, 0].set_xlabel('Polynomial Degree')
axes[1, 0].set_ylabel('R² Score')
axes[1, 0].set_title('Model Complexity vs Performance')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 5. Feature Count vs Performance
axes[1, 1].scatter(num_features, r2_scores, s=100, alpha=0.7)
for i, (x, y, degree) in enumerate(zip(num_features, r2_scores, degrees_plot)):
    axes[1, 1].annotate(f'Deg {degree}', (x, y), xytext=(5, 5), 
                       textcoords='offset points', fontsize=8)
axes[1, 1].set_xlabel('Number of Features')
axes[1, 1].set_ylabel('R² Score')
axes[1, 1].set_title('Feature Count vs Performance')
axes[1, 1].grid(True, alpha=0.3)

# 6. Performance Metrics Comparison
metrics = ['R²', 'RMSE ($000s)', 'MAE ($000s)']
linear_values = [r2_linear, rmse_linear/1000, mae_linear/1000]
poly_values = [r2_final, rmse_final/1000, mae_final/1000]

x = np.arange(len(metrics))
width = 0.35

bars1 = axes[1, 2].bar(x - width/2, linear_values, width, label='Linear', alpha=0.7)
bars2 = axes[1, 2].bar(x + width/2, poly_values, width, label='Polynomial', alpha=0.7)

axes[1, 2].set_ylabel('Score')
axes[1, 2].set_title('Performance Metrics Comparison')
axes[1, 2].set_xticks(x)
axes[1, 2].set_xticklabels(metrics)
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

# Add value labels on bars
for bar1, bar2, val1, val2 in zip(bars1, bars2, linear_values, poly_values):
    height1, height2 = bar1.get_height(), bar2.get_height()
    axes[1, 2].text(bar1.get_x() + bar1.get_width()/2., height1 + 0.01,
                    f'{val1:.3f}', ha='center', va='bottom', fontsize=8)
    axes[1, 2].text(bar2.get_x() + bar2.get_width()/2., height2 + 0.01,
                    f'{val2:.3f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

## Step 10: Summary and Key Insights

### 🎯 What We Accomplished:
1. **Explored non-linear relationships** in our housing data
2. **Created polynomial features** of different degrees
3. **Used cross-validation** to find optimal model complexity
4. **Detected and avoided overfitting** through validation curves
5. **Compared linear vs polynomial** regression performance
6. **Analyzed feature importance** in polynomial space

### 📊 Key Results:
- **Optimal Polynomial Degree**: {optimal_degree}
- **Performance Improvement**: {improvement:+.4f} R² over linear regression
- **Final R² Score**: {r2_final:.4f} ({r2_final*100:.1f}% variance explained)
- **RMSE Reduction**: ${rmse_linear - rmse_final:,.2f}

### 💡 Key Learnings:

**When to Use Polynomial Regression:**
- Data shows **curved relationships**
- Linear models **underfit** the data
- Domain knowledge suggests **non-linear effects**

**The Bias-Variance Tradeoff:**
- **Low degree**: High bias, low variance (underfitting)
- **High degree**: Low bias, high variance (overfitting)
- **Optimal degree**: Best balance for generalization

**Cross-Validation Benefits:**
- **Prevents overfitting** to training data
- **Guides model selection** objectively
- **Estimates generalization** performance

**Feature Explosion:**
- Polynomial features **grow rapidly** with degree
- **Feature scaling** becomes crucial
- **Regularization** may be needed for high degrees

### ⚠️ Potential Issues:
- **Overfitting** with high-degree polynomials
- **Numerical instability** with many features
- **Extrapolation problems** outside training range
- **Interpretability loss** with complex features

### 🚀 Next Steps:
- Learn about **Ridge and Lasso regression** for regularization
- Explore **interaction features** and **feature selection**
- Try **spline regression** for piecewise polynomials
- Study **kernel methods** for non-linear relationships

### 🤔 Questions to Explore:
- How do we handle polynomial regression with many features?
- When is polynomial regression better than other non-linear methods?
- How do we interpret complex polynomial coefficients?
- What's the relationship between polynomial regression and neural networks?

Great job mastering polynomial regression and the bias-variance tradeoff! 🎉